In [6]:
%pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib requests

In [7]:
# @title 🚀 Real-World Daily Newsletter Generator (Colab Fix)
import os
import datetime
import requests
import torch
from googleapiclient.discovery import build
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util

# --- CONFIGURATION ---
NEWS_API_KEY = "24bde5e1e1484b4d8f0c2ecbc4a04865"
# Note: In Colab, we use the built-in auth which is more robust
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

# 1. Setup Device
device = 0 if torch.cuda.is_available() else -1
print(f"Status: Running on {'GPU' if device == 0 else 'CPU'}")

# 2. Define Data Loader (Optimized for Colab)
class DataLoader:
    def __init__(self):
        self.users = [
            {
                "id": 1,
                "name": "Alex Chen",
                "role": "Software Engineer",
                "interests": ["Artificial Intelligence", "Python", "Technology"],
                "location": "us"
            }
        ]
        self.creds = None
        self.service = None

    def authenticate_google(self):
        """Authenticates using Google Colab's native login system"""
        try:
            from google.colab import auth
            from google.auth import default

            print("🔐 A login window will now open in a popup or a new tab...")
            # This handles the "headless browser" issue by using Colab's own auth bridge
            auth.authenticate_user()
            self.creds, _ = default()

            self.service = build('calendar', 'v3', credentials=self.creds)
            print("✅ Google Calendar successfully connected.")
            return True
        except Exception as e:
            print(f"❌ Auth Error: {e}")
            return False

    def fetch_calendar_events(self, max_results=5):
        if not self.service:
            if not self.authenticate_google():
                return [{"summary": "No Access", "start": "--", "description": "Auth failed"}]

        now = datetime.datetime.utcnow().isoformat() + 'Z'
        print("   -> Fetching calendar events...")

        try:
            events_result = self.service.events().list(
                calendarId='primary', timeMin=now,
                maxResults=max_results, singleEvents=True,
                orderBy='startTime'
            ).execute()
            events = events_result.get('items', [])
        except Exception as e:
            return [{"summary": "No events found (or access restricted)", "start": "--", "description": str(e)}]

        formatted_events = []
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            try:
                dt_obj = datetime.datetime.fromisoformat(start.replace('Z', '+00:00'))
                clean_time = dt_obj.strftime("%I:%M %p")
            except:
                clean_time = start

            formatted_events.append({
                "summary": event.get('summary', 'No Title'),
                "start": clean_time,
                "description": event.get('description', '')
            })
        return formatted_events if formatted_events else [{"summary": "No upcoming events", "start": "--", "description": ""}]

    def fetch_global_news(self, query=None, country='us'):
        base_url = "https://newsapi.org/v2/top-headlines"
        params = {"apiKey": NEWS_API_KEY, "country": country, "category": "technology", "pageSize": 10}
        try:
            response = requests.get(base_url, params=params)
            data = response.json()
            return [{"title": a['title'], "description": a['description'], "url": a['url']}
                    for a in data.get("articles", []) if a['title'] != "[Removed]" and a['description']]
        except:
            return []

# 3. Define ML Engine
class NewsletterEngine:
    def __init__(self):
        print("Loading ML models... (This takes a minute)")
        self.matcher = SentenceTransformer('all-MiniLM-L6-v2')
        model_name = "sshleifer/distilbart-cnn-12-6"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda" if device == 0 else "cpu")
        print("✅ Models loaded.")

    def score_relevance(self, user_interests, items):
        if not items: return []
        interest_embedding = self.matcher.encode(" ".join(user_interests), convert_to_tensor=True)
        item_embeddings = self.matcher.encode([item['description'] for item in items], convert_to_tensor=True)
        scores = util.cos_sim(interest_embedding, item_embeddings)[0]
        for i, score in enumerate(scores): items[i]['relevance_score'] = score.item()
        return sorted(items, key=lambda x: x['relevance_score'], reverse=True)

    def summarize_content(self, text):
        if not text or len(text.split()) < 25: return text
        inputs = self.tokenizer([text], max_length=1024, return_tensors="pt", truncation=True).to(self.model.device)
        summary_ids = self.model.generate(inputs["input_ids"], num_beams=2, max_length=50)
        return self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 4. Generate Newsletter Function
def generate_newsletter(user, engine, data_loader):
    events = data_loader.fetch_calendar_events()
    news = data_loader.fetch_global_news(country=user['location'])

    scored_news = engine.score_relevance(user['interests'], news)[:3]
    for article in scored_news: article['summary'] = engine.summarize_content(article['description'])

    date_str = datetime.datetime.now().strftime("%B %d, %Y")
    output = f"\n================================================\n"
    output += f"📢 DAILY BRIEFING | {date_str}\n"
    output += f"👤 User: {user['name']}\n================================================\n\n"
    output += "📅 CALENDAR:\n" + "\n".join([f"• {e['start']} - {e['summary']}" for e in events])
    output += f"\n\n🌍 TOP NEWS MATCHED TO YOUR INTERESTS:\n"
    for i, a in enumerate(scored_news):
        output += f"{i+1}. {a['title']}\n   \"{a['summary']}\"\n"
    return output

# --- EXECUTION ---
if __name__ == "__main__":
    dl = DataLoader()
    engine = NewsletterEngine()
    for user in dl.users:
        print(generate_newsletter(user, engine, dl))

Status: Running on CPU
Loading ML models... (This takes a minute)


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading weights:   0%|          | 0/358 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


✅ Models loaded.
🔐 A login window will now open in a popup or a new tab...


/tmp/ipython-input-1559374470.py:57: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + 'Z'


✅ Google Calendar successfully connected.
   -> Fetching calendar events...



📢 DAILY BRIEFING | February 11, 2026
👤 User: Alex Chen

📅 CALENDAR:
• -- - No events found (or access restricted)

🌍 TOP NEWS MATCHED TO YOUR INTERESTS:
1. Romeo Is A Dead Man slyly parodies multiverse nonsense before playing it too straight - AV Club
   "The sci-fi action game Romeo is a Dead Man delivers SUDA51's trademark strangeness before playing things a bit too straight."
2. February Update 1.15.0 - ARC Raiders
   "ARC Raiders is a multiplayer extraction adventure, set in a lethal future earth, ravaged by a mysterious mechanized threat known as ARC."
3. Nintendo keeps filling in the gaps - theverge.com
   "With games like Mario Tennis Fever and Resident Evil Requiem, Nintendo has been finding ways to flesh out the Switch 2’s library."



In [9]:
# @title  Automated Personalized Newsletter Dispatcher (v1.1)
# Project by: Md. Moshfiqur Rahman Chishti [cite: 42]
# Technical Stack: Python, Transformers, Torch, Pandas, smtplib

import os
import datetime
import requests
import torch
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from googleapiclient.discovery import build
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util

# --- CONFIGURATION ---
NEWS_API_KEY = "24bde5e1e1484b4d8f0c2ecbc4a04865"
SENDER_EMAIL = "tahseenwashere@gmail.com"  # 👈 Replace with your Gmail
SENDER_PASSWORD = "xyth vfcs dqxd qxfw" # 👈 Replace with your 16-digit App Password

# 1. Setup Device
device = 0 if torch.cuda.is_available() else -1

# 2. Data Loader with Real Email Addresses [cite: 56]
class DataLoader:
    def __init__(self):
        self.users_df = pd.DataFrame([
            {
                "id": 1,
                "name": "Alex Chen",
                "email": "jadewarrior679@gmail.com", # 👈 Dummy Address 1
                "interests": ["Artificial Intelligence", "Python", "Cloud Computing"],
                "location": "us"
            },
            {
                "id": 2,
                "name": "Sarah Miller",
                "email": "groupasshomies@gmail.com", # 👈 Dummy Address 2
                "interests": ["Leadership", "Productivity", "Agile Methodology"],
                "location": "us"
            }
        ])
        self.creds = None
        self.service = None

    def authenticate_google(self):
        try:
            from google.colab import auth
            from google.auth import default
            auth.authenticate_user()
            self.creds, _ = default()
            self.service = build('calendar', 'v3', credentials=self.creds)
            return True
        except: return False

    def fetch_calendar_events(self):
        if not self.service and not self.authenticate_google(): return []
        now = datetime.datetime.utcnow().isoformat() + 'Z'
        try:
            events_result = self.service.events().list(
                calendarId='primary', timeMin=now, maxResults=3, singleEvents=True, orderBy='startTime'
            ).execute()
            return [{"summary": e.get('summary'), "start": e['start'].get('dateTime', e['start'].get('date'))}
                    for e in events_result.get('items', [])]
        except: return []

    def fetch_global_news(self):
        base_url = "https://newsapi.org/v2/top-headlines"
        params = {"apiKey": NEWS_API_KEY, "country": "us", "category": "technology", "pageSize": 10}
        try:
            res = requests.get(base_url, params=params).json()
            return [{"title": a['title'], "description": a['description']} for a in res.get("articles", []) if a['description']]
        except: return []

# 3. ML Engine for Summarization & Matching [cite: 57, 67]
class NewsletterEngine:
    def __init__(self):
        self.matcher = SentenceTransformer('all-MiniLM-L6-v2')
        model_name = "sshleifer/distilbart-cnn-12-6"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda" if device == 0 else "cpu")

    def process_content(self, user_interests, news_items):
        if not news_items: return []
        interest_text = " ".join(user_interests)
        interest_emb = self.matcher.encode(interest_text, convert_to_tensor=True)
        news_embs = self.matcher.encode([n['description'] for n in news_items], convert_to_tensor=True)
        scores = util.cos_sim(interest_emb, news_embs)[0]

        for i, score in enumerate(scores): news_items[i]['score'] = score.item()
        top_items = sorted(news_items, key=lambda x: x['score'], reverse=True)[:3]

        for item in top_items:
            inputs = self.tokenizer(item['description'], return_tensors="pt", truncation=True).to(self.model.device)
            summary_ids = self.model.generate(inputs["input_ids"], max_length=50)
            item['summary'] = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return top_items

# 4. Email Dispatcher (Real SMTP)
class EmailDispatcher:
    @staticmethod
    def send_email(recipient_email, recipient_name, content):
        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = recipient_email
        msg['Subject'] = f"📢 Daily Briefing for {recipient_name}"

        msg.attach(MIMEText(content, 'plain'))

        try:
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls()
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.send_message(msg)
            server.quit()
            print(f" Successfully sent to: {recipient_email}")
        except Exception as e:
            print(f" Failed to send to {recipient_email}: {e}")

# 5. Execution Loop
def run_automation():
    loader = DataLoader()
    engine = NewsletterEngine()
    dispatcher = EmailDispatcher()

    for _, user in loader.users_df.iterrows():
        print(f"Generating and Sending to {user['name']}...")

        events = loader.fetch_calendar_events()
        news = loader.fetch_global_news()
        processed_news = engine.process_content(user['interests'], news)

        # Build Body
        body = f"Hello {user['name']},\n\nYour internal briefing for today:\n\n"
        body += " EVENTS:\n" + ("\n".join([f"- {e['summary']} ({e['start']})" for e in events]) if events else "- No events\n")
        body += "\n🌍 PERSONALIZED NEWS:\n"
        for i, n in enumerate(processed_news):
            body += f"{i+1}. {n['title']}\n   Summary: {n['summary']}\n\n"

        # Dispatch Real Email
        dispatcher.send_email(user['email'], user['name'], body)

if __name__ == "__main__":
    run_automation()

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading weights:   0%|          | 0/358 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Generating and Sending to Alex Chen...


/tmp/ipython-input-2639625902.py:59: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + 'Z'


 Successfully sent to: jadewarrior679@gmail.com
Generating and Sending to Sarah Miller...


 Successfully sent to: groupasshomies@gmail.com
